#### This notebook is to use Qiime2-2018.2 environment to construct phylogenetic tree, assign taxonomy, rarefaction, calculate alpha and beta diversities and generate PCoA plots

In [ ]:
### 1. construct phylogenetic tree using SEPP
qiime tools import \
--input-path mros_deblur_otus_unrare.biom \
--type 'FeatureTable[Frequency]' \
--source-format BIOMV210Format \
--output-path mros_deblur_otus_unrare.qza

qiime tools import \
--input-path reference-hit.seqs.fa \
--output-path reference-hit.seqs.qza \
--type "FeatureData[Sequence]" 

qiime fragment-insertion sepp \
  --i-representative-sequences reference-hit.seqs.qza \
  --p-threads 8 \
  --o-tree insertion-tree.qza \
  --o-placements insertion-placements.qza 

In [ ]:
### 2. assign taxonomy
qiime fragment-insertion classify-otus-experimental \
  --i-representative-sequences reference-hit.seqs.qza \
  --i-tree insertion-tree.qza \
  --i-reference-taxonomy taxonomy_gg99.qza \
  --o-classification taxonomy.qza

# visualize taxonomy
qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

# export taxonomy
qiime tools export \
  taxonomy.qza \
  --output-dir deblur_taxonomy

# taxonomy barplot 
qiime taxa barplot \
--i-table mros_deblur_otus_unrare.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file mapping_MrOS_add.txt \
--o-visualization taxonomy_barplot.qzv 

In [ ]:
### 3. rarefaction
qiime diversity alpha-rarefaction \
--i-table mros_deblur_otus_unrare.qza \
--p-max-depth 9000 \
--i-phylogeny insertion-tree.qza \
--p-metrics faith_pd \
--p-metrics observed_otus \
--p-metrics chao1 \
--p-metrics simpson_e \
--p-metrics shannon \
--m-metadata-file mapping_MrOS_add.txt \
--p-min-depth 4000 \
--o-visualization rarefyCurve.qzv 
--output-dir rarefyCurve 

# rarefy at min reads 5807
qiime feature-table rarefy \
--i-table mros_deblur_otus_unrare.qza \
--p-sampling-depth 5807 \
--o-rarefied-table mros_deblur_otus_rare5807.qza

# convert qza to biom
qiime tools export \
mros_deblur_otus_rare5807.qza \
--output-dir mros_rare5807_exp

# convert biom to txt
biom convert -i mros_rare5807_exp/feature-table.biom \
  -o mros_rare5807_exp/feature-table-rare5807.txt \
  --to-tsv

In [ ]:
### 4. compute alpha and beta diversity 
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny insertion-tree.qza \
  --i-table mros_deblur_otus_unrare.qza \
  --p-sampling-depth 5807 \
  --m-metadata-file mapping_MrOS_add.txt \
  --output-dir core-metrics-results

# export    
qiime tools export \
  core-metrics-results/faith_pd_vector.qza \
  --output-dir alpha_PD

qiime tools export \
  core-metrics-results/shannon_vector.qza \
  --output-dir alpha_Shannon

qiime tools export \
  core-metrics-results/observed_otus_vector.qza \
  --output-dir alpha_ObsOTU

qiime tools export \
  core-metrics-results/evenness_vector.qza \
  --output-dir alpha_Evenness

In [ ]:
### 5. PCoA plot of beta diversities
qiime tools export \
  unweighted_unifrac_pcoa_results.qza \
  --output-dir unweighted_unifrac_pcoa_results

qiime tools export \
  unweighted_unifrac_distance_matrix.qza \
  --output-dir unweighted_unifrac_distance

qiime tools export \
  weighted_unifrac_pcoa_results.qza \
  --output-dir weighted_unifrac_pcoa_results

qiime tools export \
  weighted_unifrac_distance_matrix.qza \
  --output-dir weighted_unifrac_distance